In [1]:
import geopandas as gpd
import os
import zipfile
import osmnx as ox
import pandas as pd
import shapely.wkb as wkblib
from shapely.geometry import Point, LineString, Polygon
import geopandas as gpd
import glob
import pickle

In [2]:
!pip install osmium

In [ ]:
!conda install -c conda-forge gdal -y

In [9]:
import osmium
from osgeo import gdal
from osgeo import ogr
import os
import geopandas as gpd
import pandas as pd
import pickle
from osgeo import ogr
from shapely.wkb import loads as wkblib
import osmium as osm
import re

For the time series analysis on EV charging demand, amenities that might impact demand could include:

1. Parking: The presence of parking amenities could indicate potential areas where people leave their cars, and therefore might need charging stations.

2. Transportation Hubs (Bus Station, Train Station): These areas could have high vehicle turnover and might benefit from charging stations.

3. Commercial Areas (Shopping Malls, Supermarkets, Restaurants): These are places where people typically spend time and could use charging facilities.

4. Lodging (Hotels): Overnight guests with EVs will likely need charging facilities.

5. Educational Institutions (Schools, Universities): These places have high parking and potential charging demand during the day.

6. Recreational Facilities (Cinema, Theatre): People usually spend two to three hours in these places, providing a good opportunity for EV charging.

To incorporate these amenities into your current dataset, you can calculate the count or density of each amenity type around each charging point within a certain radius. This will provide additional context about each charging point's location. You could also calculate the minimum distance to each amenity type from each charging point, as an indicator of convenience/accessibility.

In [10]:
# # Define features
# features = {
#     'amenity:fuel': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:parking': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:bus_station': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:train_station': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'shop:mall': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'shop:supermarket': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:restaurant': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'building:hotel': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:school': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:university': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:cinema': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
#     'amenity:theatre': gpd.GeoDataFrame(columns=['geometry', 'osm_time']),
# }

# # Initialize WKB factory
# wkbfab = osm.geom.WKBFactory()

# # Define handler
# class FeatureHandler(osm.SimpleHandler):
#     def __init__(self, features, year):
#         osm.SimpleHandler.__init__(self)
#         self.features = features
#         self.year = year

#     def node(self, n):
#         for tag in n.tags:
#             feature = f"{tag.k}:{tag.v}"
#             if feature in self.features:
#                 wkb = wkbfab.create_point(n)
#                 point = wkblib(wkb, hex=True)
#                 new_row = gpd.GeoDataFrame({'geometry': [point], 'osm_time': [self.year]})
#                 self.features[feature] = pd.concat([self.features[feature], new_row], ignore_index=True)

# # Define function to read OSM file
# def read_osm_pbf(file_path, features, year):
#     handler = FeatureHandler(features, year)
#     handler.apply_file(file_path)
#     print("Features after reading:")
#     for key, value in features.items():
#         print(f"Key: {key}, Value type: {type(value)}, Value length: {len(value)}")
#     return features

# # Define function to save features
# def save_features(features, year):
#     cwd = os.getcwd()
#     with open(os.path.join(cwd, '..', 'data', 'interim', f'features_{year}.pickle'), 'wb') as handle:
#         pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     # Load and inspect immediately
#     with open(os.path.join(cwd, '..', 'data', 'interim', f'features_{year}.pickle'), 'rb') as handle:
#         loaded_object = pickle.load(handle)
#         print(f"Loaded object type for year {year}: {type(loaded_object)}")
#         for key, value in loaded_object.items():
#             print(f"Key: {key}, Value type: {type(value)}, Value length: {len(value)}")



In [ ]:
## Process files
# cwd = os.getcwd()
# files = ['Scotland-2016.osm.pbf', 'Scotland-2017.osm.pbf', 'Scotland-2018.osm.pbf', 'Scotland-2019.osm.pbf']  

# for file in files:
#     year = int(file.split('-')[1].split('.')[0])
#     file_path = os.path.join(cwd, '..', 'data', 'raw', file)
#     if os.path.exists(file_path):
#         print(f"Processing file for year {year}...")
#         features = read_osm_pbf(file_path, features, year)
#         save_features(features, year)
#         print(f"Features for year {year} saved successfully.")
#     else:
#         print(f"No data found for year {year}.")

In [11]:
# Initialize an empty GeoDataFrame to store all features
all_features_gdf = gpd.GeoDataFrame()

# Process files
cwd = os.getcwd()
files = ['features_2016.pickle', 'features_2017.pickle', 'features_2018.pickle', 'features_2019.pickle']

In [16]:
def process_df(df, feature_name):
    # Convert "amenity:fuel" into "fuel_station"
    column_name = re.sub(r'[^a-zA-Z ]', '', feature_name.split(":")[-1]) + "_station"
    
    # Create a new column for the feature
    df[column_name] = 1
    
    # drop unnecessary columns
    if 'osm_time' in df.columns:
        df = df.drop(['osm_time'], axis=1)
    
    return df

for file in files:
    year = int(file.split('_')[1].split('.')[0])
    file_path = os.path.join(cwd, '..', 'data', 'interim', file)
    if os.path.exists(file_path):
        print(f"Processing file for year {year}...")
        # Load the object
        with open(file_path, 'rb') as handle:
            loaded_object = pickle.load(handle)
        print(f"Loaded object type for year {year}: {type(loaded_object)}")
        # Process only if loaded object is a dictionary
        if isinstance(loaded_object, dict):
            # Append 'osm_year' column to each GeoDataFrame inside the dictionary
            for feature, gdf in loaded_object.items():
                print(f"Key: {feature}, Value type: {type(gdf)}, Value length: {len(gdf)}")
                gdf['osm_year'] = year
                gdf = process_df(gdf, feature)  # Pass the feature as an argument here
                # Concatenate the GeoDataFrame to the main GeoDataFrame
                all_features_gdf = pd.concat([all_features_gdf, gdf])
            print(f"Features for year {year} processed successfully.")
        else:
            print(f"Skipped processing for year {year} as the loaded object is not a dictionary.")
    else:
        print(f"No data found for year {year}.")

# Save the `all_features_gdf` GeoDataFrame to a new pickle file
with open(os.path.join(cwd, '..', 'data', 'interim', 'all_features.pickle'), 'wb') as handle:
    pickle.dump(all_features_gdf, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("All features saved successfully.")


Processing file for year 2016...
Loaded object type for year 2016: <class 'dict'>
Key: amenity:fuel, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 546
Key: amenity:parking, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 3118
Key: amenity:bus_station, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 22
Key: amenity:train_station, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 0
Key: shop:mall, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 4
Key: shop:supermarket, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 378
Key: amenity:restaurant, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 1479
Key: building:hotel, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 1
Key: amenity:school, Value type: <class 'geopandas.geodataframe.GeoDataFrame'>, Value length: 485
Key: amenity:university, Valu

In [22]:
all_features_gdf.head()

,geometry,osm_year,fuel_station,parking_station,busstation_station,trainstation_station,mall_station,supermarket_station,restaurant_station,hotel_station,school_station,university_station,cinema_station,theatre_station
0,POINT (-4.34409 55.93436),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-3.72159 56.69738),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (-3.16965 56.19748),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (-3.29724 55.89943),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (-3.28774 55.92326),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
all_features_gdf[700:705]

,geometry,osm_year,fuel_station,parking_station,busstation_station,trainstation_station,mall_station,supermarket_station,restaurant_station,hotel_station,school_station,university_station,cinema_station,theatre_station
154,POINT (-4.28347 56.27753),2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,POINT (-4.75408 56.56805),2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,POINT (-4.80973 56.63659),2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,POINT (-4.96470 56.66313),2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,POINT (-4.99288 56.87146),2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
all_features_gdf.to_csv(os.path.join(cwd, '..', 'data', 'interim', 'all_features_gdf.csv'))